# DeepFM

이번 실습에서는 DeepFM 모델을 이해하고 구현해보겠습니다.  

DeepFM 모델은 Factorization machines와 neural network를 합친 모델로, Wide & Deep model과 유사하지만, feature engineering이 필요하지 않다는 특징을 가지고 있습니다.  
<br/>
사용자가 영화에 대해 Rating한 데이터, 영화의 장르 데이터를 이용하여 Train/Test data를 생성한 다음, Train data로 학습한 모델을 Test data에 대해 평가해봅니다.   
사용한 데이터는 Implicit feedback data로, 사용자가 시청한 영화(Positive instances)는 rating = 1로 기록됩니다. 따라서 시청하지 않은 영화에 대해 각 유저별로 Negative instances sampling을 진행합니다.   
<br/>
**구현에 앞서, DeepFM 논문을 꼭 읽어보시길 권장합니다.**

* 참고  
    - DeepFM: A Factorization-Machine based Neural Network for CTR Prediction (https://arxiv.org/pdf/1703.04247.pdf)  
    - Wide & Deep Learning for Recommender Systems (https://arxiv.org/pdf/1606.07792.pdf)
    - Factorization Machines (https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=5694074)
    - https://d2l.ai/chapter_recommender-systems/deepfm.html

# Modules

In [1]:
import csv
import numpy as np
import pandas as pd
from collections import Counter
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data preprocessing
0. Dataset 다운로드  
<br/>
1. Rating df 생성  
rating 데이터(train_ratings.csv)를 불러와 [user, item, rating]의 컬럼으로 구성된 데이터 프레임을 생성합니다.   
<br/>
2. Genre df 생성   
genre 정보가 담긴 데이터(genres.tsv)를 불러와 genre이름을 id로 변경하고, [item, genre]의 컬럼으로 구성된 데이터 프레임을 생성합니다.    
<br/>
3. Negative instances 생성   
rating 데이터는 implicit feedback data(rating :0/1)로, positive instances로 구성되어 있습니다. 따라서 rating이 없는 item중 negative instances를 뽑아서 데이터에 추가하게 됩니다.   
<br/>
4. Join dfs   
rating df와 genre df를 join하여 [user, item, rating, genre]의 컬럼으로 구성된 데이터 프레임을 생성합니다.   
<br/>
5. zero-based index로 mapping   
Embedding을 위해서 user,item,genre를 zero-based index로 mapping합니다.
    - user : 0-31359
    - item : 0-6806
    - genre : 0-17  
<br/>
6. feature matrix X, label tensor y 생성   
[user, item, genre] 3개의 field로 구성된 feature matrix를 생성합니다.   
<br/>
7. data loader 생성

## 데이터 다운로드
이곳에 대회 사이트(AI Stages)에 있는 data의 URL을 입력해주세요. 
- 데이터 URL은 변경될 수 있습니다.
- 예) `!wget https://aistages-prod-server-public.s3.amazonaws.com/app/Competitions/000176/data/data.tar.gz`

In [32]:
# 0. Dataset 다운로드
#!wget https://aistages-prod-server-public.s3.amazonaws.com/app/Competitions/000179/data/data.tar.gz
#!tar -xf data.tar.gz

In [2]:
# 1. Rating df 생성
rating_data = "./data/train/train_ratings.csv"

raw_rating_df = pd.read_csv(rating_data)#user,item,time
raw_rating_df
raw_rating_df['rating'] = 1.0 # implicit feedback
raw_rating_df.drop(['time'],axis=1,inplace=True)
print("Raw rating df")
print(raw_rating_df)#user,item,rate

users = set(raw_rating_df.loc[:, 'user'])#전체 user 집합
print(len(users))
items = set(raw_rating_df.loc[:, 'item'])#전체 item 집합
print(len(items))
#2. Genre df 생성
genre_data = "./data/train/genres.tsv"

raw_genre_df = pd.read_csv(genre_data, sep='\t')
raw_genre_df = raw_genre_df.drop_duplicates(subset=['item']) #item별 하나의 장르만 남도록 drop
# print(raw_genre_df)

genre_dict = {genre:i for i, genre in enumerate(set(raw_genre_df['genre']))}
raw_genre_df['genre']  = raw_genre_df['genre'].map(lambda x : genre_dict[x]) #genre id로 변경
print("Raw genre df - changed to id")
print(raw_genre_df)#item, genre

Raw rating df
           user   item  rating
0            11   4643     1.0
1            11    170     1.0
2            11    531     1.0
3            11    616     1.0
4            11   2140     1.0
...         ...    ...     ...
5154466  138493  44022     1.0
5154467  138493   4958     1.0
5154468  138493  68319     1.0
5154469  138493  40819     1.0
5154470  138493  27311     1.0

[5154471 rows x 3 columns]
31360
6807
Raw genre df - changed to id
         item  genre
0         318     16
2        2571     17
5        2959     17
9         296      9
13        356      9
...       ...    ...
15925   73106      9
15926  109850     17
15929    8605     17
15931    3689      9
15932    8130     11

[6807 rows x 2 columns]


In [3]:
# #2. years df 생성
# year_data = "./data/train/years.tsv"

# raw_year_df = pd.read_csv(year_data, sep='\t')

# #print(raw_year_df)
# raw_year_df = raw_year_df.append({'item':6987,'year':1920},ignore_index=True)
# raw_year_df = raw_year_df.append({'item':3310,'year':1921},ignore_index=True)
# raw_year_df = raw_year_df.append({'item':7065,'year':1915},ignore_index=True)
# raw_year_df = raw_year_df.append({'item':7243,'year':1916},ignore_index=True)
# raw_year_df = raw_year_df.append({'item':8511,'year':1917},ignore_index=True)
# raw_year_df = raw_year_df.append({'item':6988,'year':1919},ignore_index=True)
# raw_year_df = raw_year_df.append({'item':32898,'year':1902},ignore_index=True)
# raw_year_df = raw_year_df.append({'item':119145,'year':2015},ignore_index=True)

# year_dict = {year:i for i, year in enumerate(set(raw_year_df['year']))}
# raw_year_df['year']  = raw_year_df['year'].map(lambda x : year_dict[x]) #genre id로 변경

# print(raw_year_df)

In [3]:
# 3. Negative instance 생성
print("Create Nagetive instances")
num_negative = 50
user_group_dfs = list(raw_rating_df.groupby('user')['item'])
first_row = True
user_neg_dfs = pd.DataFrame()

for u, u_items in tqdm(user_group_dfs):
    u_items = set(u_items)
    i_user_neg_item = np.random.choice(list(items - u_items), num_negative, replace=False)
    #user가 보지 않은 영화 중에 50개를 랜덤으로 고름
    i_user_neg_df = pd.DataFrame({'user': [u]*num_negative, 'item': i_user_neg_item, 'rating': [0]*num_negative})
    if first_row == True:
        user_neg_dfs = i_user_neg_df
        first_row = False
    else:
        user_neg_dfs = pd.concat([user_neg_dfs, i_user_neg_df], axis = 0, sort=False)

raw_rating_df = pd.concat([raw_rating_df, user_neg_dfs], axis = 0, sort=False)

# 4. Join dfs
joined_rating_df = raw_rating_df
joined_rating_df = pd.merge(raw_rating_df, raw_genre_df, left_on='item', right_on='item', how='inner')
#joined_rating_df = pd.merge(joined_rating_df, raw_year_df, left_on='item', right_on='item', how='inner')

# print("Joined rating df")
# print(joined_rating_df)

# 5. user, item을 zero-based index로 mapping
users = list(set(joined_rating_df.loc[:,'user']))
users.sort()
items =  list(set((joined_rating_df.loc[:, 'item'])))
items.sort()
genres =  list(set((joined_rating_df.loc[:, 'genre'])))
genres.sort()
# years = list(set((joined_rating_df.loc[:,'year'])))
# years.sort()

if len(users)-1 != max(users):
    users_dict = {users[i]: i for i in range(len(users))}
    joined_rating_df['user']  = joined_rating_df['user'].map(lambda x : users_dict[x])
    users = list(set(joined_rating_df.loc[:,'user']))
    
if len(items)-1 != max(items):
    items_dict = {items[i]: i for i in range(len(items))}
    joined_rating_df['item']  = joined_rating_df['item'].map(lambda x : items_dict[x])
    items =  list(set((joined_rating_df.loc[:, 'item'])))

joined_rating_df = joined_rating_df.sort_values(by=['user'])
joined_rating_df.reset_index(drop=True, inplace=True)

data = joined_rating_df
print("Data")
print(data)

n_data = len(data)
n_user = len(users)
n_item = len(items)
n_genre = len(genres)
#n_year = len(years)

print("# of data : {}\n# of users : {}\n# of items : {}\n# of genres : {}".format(n_data, n_user, n_item, n_genre))

Create Nagetive instances


100%|██████████| 31360/31360 [04:58<00:00, 104.99it/s]


Data
          user  item  rating  genre
0            0  2505     1.0     17
1            0  3253     1.0     17
2            0  2425     1.0      6
3            0  6358     0.0     10
4            0  4059     1.0     17
...        ...   ...     ...    ...
6722466  31359  4882     1.0      6
6722467  31359   229     1.0     17
6722468  31359  2485     1.0      6
6722469  31359  3646     0.0      9
6722470  31359   585     1.0      9

[6722471 rows x 4 columns]
# of data : 6722471
# of users : 31360
# of items : 6807
# of genres : 18


In [4]:
#6. feature matrix X, label tensor y 생성
user_col = torch.tensor(data.loc[:,'user'])
item_col = torch.tensor(data.loc[:,'item'])
genre_col = torch.tensor(data.loc[:,'genre'])
#year_col = torch.tensor(data.loc[:,'year'])

# offsets = [0,n_user]
# for col, offset in zip([user_col, item_col], offsets):
#      col += offset

offsets = [0, n_user, n_user+n_item]
for col, offset in zip([user_col, item_col, genre_col], offsets):
    col += offset

# offsets = [0, n_user, n_user+n_item,n_user+n_item+n_genre]
#for col, offset in zip([user_col, item_col, genre_col, year_col], offsets):
#    col += offset


#X = torch.cat([user_col.unsqueeze(1), item_col.unsqueeze(1)], dim=1)
X = torch.cat([user_col.unsqueeze(1), item_col.unsqueeze(1), genre_col.unsqueeze(1)], dim=1)
#X = torch.cat([user_col.unsqueeze(1), item_col.unsqueeze(1), genre_col.unsqueeze(1),year_col.unsqueeze(1)], dim=1)
y = torch.tensor(list(data.loc[:,'rating']))
print(y.shape)

#7. data loader 생성
class RatingDataset(Dataset):
    def __init__(self, input_tensor, target_tensor):
        self.input_tensor = input_tensor.long()
        self.target_tensor = target_tensor.long()

    def __getitem__(self, index):
        return self.input_tensor[index], self.target_tensor[index]

    def __len__(self):
        return self.target_tensor.size(0)


dataset = RatingDataset(X, y)
train_ratio = 0.9

train_size = int(train_ratio * len(data))
test_size = len(data) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

torch.Size([6722471])


   # Model architecture (DeepFM)
   DeepFM 모델은 1) FM component와  2) Deep component가 병렬적으로 결합되어 있습니다. 구조는 다음과 같습니다.
<img src='https://drive.google.com/uc?id=1vwcxUJQTIsg5QH9CuH5PcUEfExhToUHR'>  
각 구조는 다음과 같습니다.  
   **1. FM component**  
       FM component는 우리가 아는 2-way Factorization machines(degree=2)입니다. FM은 variables 간의 interaction을 다음과 같이 모델링 합니다.   
     **<center> equation (1) </center>**
   $$\hat{y}(x):=w_0 + \sum_{i=1}^{n}w_ix_i + \sum_{i=1}^{n}\sum_{j=i+1}^{n}<\mathbf{v}_i,\mathbf{v}_j>x_ix_j$$   
   이때, 세번째 interaction term을 전개하여 다음과 같이 쓸 수 있습니다.(논문 참고)  
   구현 코드는 전개된 식을 바탕으로 합니다.   
     **<center> equation (2)> </center>**
   $$\sum_{i=1}^{n}\sum_{j=i+1}^{n}<\mathbf{v}_i,\mathbf{v}_j>x_ix_j = \frac{1}{2}\sum_{f=1}^{k}((\sum_{i=1}^{n}v_{i,f}x_i)^2-\sum_{i=1}^{n}v_{i,f}^2x_i^2)$$   
           
   **2. Deep component**  
       Deep component는 MLP Layers로 구성되어 있습니다.   
       구현 코드는 Input dimension이 30-20-10인 3 layer MLP 구조입니다.
  
   

# DeepFM

In [5]:
class DeepFM(nn.Module):
    def __init__(self, input_dims, embedding_dim, mlp_dims, drop_rate=0.1):
        super(DeepFM, self).__init__()
        total_input_dim = int(sum(input_dims)) # n_user + n_movie + n_genre

        # Fm component의 constant bias term과 1차 bias term
        self.bias = nn.Parameter(torch.zeros((1,)))
        self.fc = nn.Embedding(total_input_dim, 1)
        
        self.embedding = nn.Embedding(total_input_dim, embedding_dim) 
        self.embedding_dim = len(input_dims) * embedding_dim

        mlp_layers = []
        for i, dim in enumerate(mlp_dims):
            if i==0:
                mlp_layers.append(nn.Linear(self.embedding_dim, dim))
            else:
                mlp_layers.append(nn.Linear(mlp_dims[i-1], dim)) #TODO 1 : linear layer를 넣어주세요.
            mlp_layers.append(nn.ReLU(True))
            mlp_layers.append(nn.Dropout(drop_rate))
        mlp_layers.append(nn.Linear(mlp_dims[-1], 1))
        self.mlp_layers = nn.Sequential(*mlp_layers)

    def fm(self, x):
        # x : (batch_size, total_num_input)
        embed_x = self.embedding(x)

        fm_y = self.bias + torch.sum(self.fc(x), dim=1)
        square_of_sum = torch.sum(embed_x, dim=1) ** 2         #TODO 2 : torch.sum을 이용하여 square_of_sum을 작성해주세요(hint : equation (2))
        sum_of_square = torch.sum(embed_x ** 2, dim=1)         #TODO 3 : torch.sum을 이용하여 sum_of_square을 작성해주세요(hint : equation (2))
        fm_y += 0.5 * torch.sum(square_of_sum - sum_of_square, dim=1, keepdim=True)
        return fm_y
    
    def mlp(self, x):
        embed_x = self.embedding(x)
        
        inputs = embed_x.view(-1, self.embedding_dim)
        mlp_y = self.mlp_layers(inputs)
        return mlp_y

    def forward(self, x):
        embed_x = self.embedding(x)
        #fm component
        fm_y = self.fm(x).squeeze(1)
        
        #deep component
        mlp_y = self.mlp(x).squeeze(1)
        
        y = torch.sigmoid(fm_y + mlp_y)
        return y


# Training

In [30]:
device = torch.device('cuda')
#input_dims = [n_user,n_item]
input_dims = [n_user, n_item, n_genre]
#input_dims = [n_user, n_item, n_genre,n_year]
embedding_dim = 10
model = DeepFM(input_dims, embedding_dim, mlp_dims=[8, 4, 1]).to(device)#30,20,10
bce_loss = nn.BCELoss() # Binary Cross Entropy loss
lr, num_epochs = 0.001, 10
optimizer = optim.Adam(model.parameters(), lr=lr)

for e in tqdm(range(num_epochs)) :
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        model.train()
        optimizer.zero_grad()
        output = model(x)
        loss = bce_loss(output, y.float())
        loss.backward()
        optimizer.step()
        

100%|██████████| 10/10 [11:49<00:00, 70.96s/it]


# Evaluation
평가는 모델이 postive instance에 대해 0.5이상, negative instance에 대해 0.5미만의 값을 예측한 Accuracy를 측정하여 진행됩니다.

In [31]:
correct_result_sum = 0
final = []
for x, y in test_loader:
    x, y = x.to(device), y.to(device)
    model.eval()
    output = model(x)
    result = torch.round(output)
    final.append(result)
    correct_result_sum += (result == y).sum().float()
acc = correct_result_sum/len(test_dataset)*100
print("Final Acc : {:.2f}%".format(acc.item()))
#32,16,4->87.15
#32,8,4->87.22
#16,8,4->87.23
#16,4,1 ->87.88
#8,4,1 -> 88.37

Final Acc : 88.37%


In [32]:
real_users = list(set(raw_rating_df.loc[:, 'user']))#전체 user 집합
real_items = list(set(raw_rating_df.loc[:, 'item']))#전체 item 집합
real_users.sort()
real_items.sort()
#user encoding
user_encoding = pd.DataFrame({'real user':real_users})
user_encoding['encoded user'] = list(range(len(user_encoding['real user'])))
#item encoding
item_encoding = pd.DataFrame({'real item':real_items})
item_encoding['encoded item'] = list(range(len(item_encoding['real item'])))


In [33]:
item_encoding

,real item,encoded item
0,1,0
1,2,1
2,3,2
3,4,3
4,5,4
...,...,...
6802,118700,6802
6803,118900,6803
6804,118997,6804
6805,119141,6805


In [34]:
data_already_seen = data[data['rating']==1]
data_already_seen

,user,item,rating,genre
0,0,2505,1.0,8
1,0,3691,1.0,8
2,0,1765,1.0,1
3,0,1564,1.0,1
4,0,1667,1.0,8
...,...,...,...,...
6722463,31359,421,1.0,6
6722464,31359,658,1.0,8
6722465,31359,5130,1.0,11
6722466,31359,3716,1.0,6


In [35]:
from pandas import Series, DataFrame
import itertools
data_new = pd.DataFrame({'user': list(itertools.chain(*list([i]*6807 for i in range(31360)))),
            'item': list(i for i in range(6807))*31360,
            'rating': [None]*31360*6807
            })
data_new = data_new.append(data_already_seen)
data_new = data_new.drop_duplicates(subset = ['user','item'],keep=False)
#data_new = pd.merge(data_new, raw_genre_df, left_on='item', right_on='item', how='inner')

/tmp/ipykernel_1206/1219149434.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_new = data_new.append(data_already_seen)


In [36]:
raw_genre_df = pd.merge(item_encoding,raw_genre_df,left_on='real item',right_on='item')
raw_genre_df

,real item,encoded item,item,genre
0,1,0,1,11
1,2,1,2,11
2,3,2,3,6
3,4,3,4,6
4,5,4,5,6
...,...,...,...,...
6802,118700,6802,118700,5
6803,118900,6803,118900,5
6804,118997,6804,118997,15
6805,119141,6805,119141,8


In [37]:
raw_genre_df=raw_genre_df.drop('real item',axis =1)
raw_genre_df=raw_genre_df.drop('item',axis =1)
raw_genre_df

,encoded item,genre
0,0,11
1,1,11
2,2,6
3,3,6
4,4,6
...,...,...
6802,6802,5
6803,6803,5
6804,6804,15
6805,6805,8


In [20]:
# raw_year_df = pd.merge(item_encoding,raw_year_df,left_on='real item',right_on='item',how='inner')
# raw_year_df=raw_year_df.drop('real item',axis =1)
# raw_year_df=raw_year_df.drop('item',axis =1)
# raw_year_df

,encoded item,year
0,0,80
1,1,80
2,2,80
3,3,80
4,4,80
...,...,...
6802,6802,99
6803,6803,99
6804,6804,99
6805,6805,99


In [38]:
data_new = pd.merge(data_new, raw_genre_df, left_on='item', right_on='encoded item', how='inner')
#data_new = pd.merge(data_new,raw_year_df,left_on='item',right_on='encoded item',how = 'inner')
data_new.sort_values(by='user',axis=0,inplace=True)

In [39]:
data_new

,user,item,rating,genre_x,encoded item,genre_y
0,0,1,None,NaN,1,11
10946716,0,414,None,NaN,414,7
189838063,0,6547,None,NaN,6547,9
142831002,0,4969,None,NaN,4969,6
68006008,0,2413,None,NaN,2413,6
...,...,...,...,...,...,...
45646820,31359,1644,None,NaN,1644,6
181093151,31359,6264,None,NaN,6264,14
131214685,31359,4562,None,NaN,4562,15
131058246,31359,4556,None,NaN,4556,8


In [42]:
data_new = data_new.drop('genre_x',axis = 1)
#data_new = data_new.drop('year_x',axis = 1)
data_new = data_new.drop('encoded item',axis = 1)
#data_new = data_new.drop('encoded item_y',axis = 1)
data_new = data_new.drop('rating',axis = 1)

KeyError: "['genre_x'] not found in axis"

In [45]:
data_new

,user,item,genre_y
0,0,1,11
10946716,0,414,7
189838063,0,6547,9
142831002,0,4969,6
68006008,0,2413,6
...,...,...,...
45646820,31359,1644,6
181093151,31359,6264,14
131214685,31359,4562,15
131058246,31359,4556,8


In [46]:
real_items = np.array(real_items)

In [47]:
counts = data_new.groupby('user').count()['item']
new_counts = np.cumsum(counts)
new_counts = np.append(np.array([0]),new_counts)
new_counts

array([        0,      6431,     13058, ..., 208299817, 208306556,
       208313049])

In [48]:
item_output=[]
model.eval()
for i in tqdm(range(len(users))):#31360
    uig = data_new.iloc[new_counts[i]:new_counts[i+1]]
    predict_item_ids = uig.iloc[:,1].values
    uig = uig.values
    uig = torch.from_numpy(uig)
    uig = uig.to(device)
    prediction = model(uig)
    del uig
    prediction = prediction.data.detach().cpu().numpy()
    ind = np.where(prediction == 1)
    #ind = prediction.argpartition(-10)[-10:]
    torch.cuda.empty_cache()
    if len(list(real_items[predict_item_ids[ind]])) <=10:
        item_output.append(list(predict_item_ids[prediction.argpartition(-10)[-10:]]))
    else:
        item_output.append(list(predict_item_ids[ind]))
    #item_output.extend(real_items[predict_item_ids[ind]])


  0%|          | 109/31360 [00:00<00:56, 556.70it/s]

0
0
0
0
57
0
4
0
0
3
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
9
0
0
0
0
4
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
1
0
0
3
0
4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
5


  1%|          | 236/31360 [00:00<00:51, 604.45it/s]

0
0
0
0
0
0
0
0
2
0
0
0
0
0
5
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
15
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
2
0
4
0
0
0
0
0
0
2
0
0
3
0
1
0
0
0
0
0
0
0
0
0
0
0


  1%|          | 362/31360 [00:00<00:50, 616.67it/s]

...

0
0
0
0
0
0
4
0
0
0
0
0
0
0
0
0
0
0
0
5
3
0
0
2
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
1
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
11
12
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0


100%|█████████▉| 31322/31360 [00:49<00:00, 647.52it/s]

0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
1
0
0
0
0
0
0
0
2
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
1
0
1
0
2
0
0
0
0
0
2
0
0
4
0
0
0
0
0
1
0
0
0
1
3
0
0
0
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
9
0
0
0
0
2
0
0
0
0
0
0
3
0
0
0
0
0
1
0
0
1
0
0
0
0
0
1
0
0
0
2
0
0
0
1
0
0
0
0
0


100%|██████████| 31360/31360 [00:49<00:00, 631.42it/s]

4
0
0
0
1
0
0
0
2
0
0
0
0
0
0
0
0
0
0
1
0
0
0


In [54]:
freq = data['item'].value_counts(ascending=True)
freq

1390      223
6785      227
2278      236
4883      242
4308      243
        ...  
220     17444
193     18270
179     18307
1642    18524
1435    19802
Name: item, Length: 6807, dtype: int64

In [55]:
item_output_final = []
for i in tqdm(range(len(item_output))):
    if len(item_output[i])!=10:
        item_output[i].sort(key=lambda x : freq[x],reverse=True)
        item_output_final.extend(real_items[item_output[i][:10]])
    else:
        item_output_final.extend(real_items[item_output[i]])
        
#313600
#real_item에 대입해야함


100%|██████████| 31360/31360 [00:00<00:00, 208426.16it/s]


In [56]:
len(item_output_final)
#313600

313600

In [57]:

submission = pd.read_csv("/opt/ml/input/data/eval/sample_submission.csv")

In [59]:
submission["item"] = item_output_final


In [ ]:
submission.to_csv("./deepFM_3.csv", index=False)

In [22]:
prediction

array([0.999907  , 0.11939085, 0.99889934, ..., 0.99506   , 0.05111589,
       0.00178147], dtype=float32)

In [23]:
(prediction==1).sum()

65

In [32]:
#여기서부터는 앙상블 코드입니다
k = pd.read_csv('lightFM_ensemble_20.csv')
k = pd.merge(k, item_encoding, left_on='item', right_on='real item', how='inner')
k = k.sort_values(by=['user'])
k
#627200

,user,item,real item,encoded item
0,11,72998,72998,5936
1,82,72998,72998,5936
2,285,72998,72998,5936
3,304,72998,72998,5936
4,383,72998,72998,5936
...,...,...,...,...
627195,138011,31431,31431,4509
627196,138034,2033,2033,1104
627197,138325,25945,25945,4176
627198,138397,3802,3802,2091


In [48]:
item_output_final = []

for i in tqdm(range(31360)):
    items=list(k['encoded item'][i*20:i*20+20])
    items.sort(key=lambda x : freq[x],reverse=True)
    item_output_final.extend(real_items[items[:10]])
    #print(len(real_items[item_output[:10]]))

100%|██████████| 31360/31360 [00:04<00:00, 7625.01it/s]
